In [20]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import pandas as pd
import numpy as np

import load_data

### Simple softmax

In [55]:
# Input 20 features (60% PCA variance), output 32 tissue types
x = tf.placeholder(tf.float32, [None, 20])
W = tf.Variable(tf.zeros([20, 32]))
b = tf.Variable(tf.zeros([32]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [56]:
y_ = tf.placeholder(tf.float32, [None, 32])
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)

In [57]:
tcga = load_data.read_data_sets("./data/mRNA_PCA_20_components.csv", "label_tissue")
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
init.run()
epoch = 0
while tcga.train.epochs_completed < 20:
    batch_xs, batch_ys = tcga.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    if tcga.train.epochs_completed > epoch and epoch % 5 == 0:
        epoch += 5
        correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        train_accuracy = sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys})
        test_accuracy = sess.run(accuracy, feed_dict={x: tcga.test.X, y_: tcga.test.y})
        print("epoch={0}, train accuracy={1}, test acurracy={2}".format(
            epoch, train_accuracy, test_accuracy))

epoch=5, train accuracy=0.8700000047683716, test acurracy=0.8535681366920471
epoch=10, train accuracy=0.8899999856948853, test acurracy=0.8832252025604248
epoch=15, train accuracy=0.9399999976158142, test acurracy=0.8841519951820374
epoch=20, train accuracy=0.8600000143051147, test acurracy=0.8878591060638428


### One hidden layer

In [66]:
def weight_variable(shape):
    initial=tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [67]:
# fully connected layer
x = tf.placeholder(tf.float32, [None, 20])
W1 = weight_variable([20, 10])
b1 = bias_variable([10])
fc1 = tf.nn.relu(tf.matmul(x, W1) + b1)

W2 = weight_variable([10, 32])
b2 = bias_variable([32])
y_nn = tf.matmul(fc1, W2) + b2

In [70]:
# loss and accuracy
y_ = tf.placeholder(tf.float32, [None, 32])
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_nn))
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_nn, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [71]:
# training
tcga = load_data.read_data_sets("./data/mRNA_PCA_20_components.csv", "label_tissue")
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
init.run()
epoch = 0
while tcga.train.epochs_completed < 20:
    batch_xs, batch_ys = tcga.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    if tcga.train.epochs_completed > epoch:
        epoch += 1
        correct_prediction = tf.equal(tf.argmax(y_nn,1), tf.argmax(y_,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        train_accuracy = sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys})
        test_accuracy = sess.run(accuracy, feed_dict={x: tcga.test.X, y_: tcga.test.y})
        print("epoch={0}, train accuracy={1}, test acurracy={2}".format(
            epoch, train_accuracy, test_accuracy))

epoch=1, train accuracy=0.6000000238418579, test acurracy=0.5514365434646606
epoch=2, train accuracy=0.7799999713897705, test acurracy=0.7275254726409912
epoch=3, train accuracy=0.8700000047683716, test acurracy=0.7868396639823914
epoch=4, train accuracy=0.8500000238418579, test acurracy=0.8378127813339233
epoch=5, train accuracy=0.7900000214576721, test acurracy=0.840593159198761
epoch=6, train accuracy=0.8399999737739563, test acurracy=0.8591288328170776
epoch=7, train accuracy=0.8399999737739563, test acurracy=0.8619091510772705
epoch=8, train accuracy=0.800000011920929, test acurracy=0.8693234324455261
epoch=9, train accuracy=0.8899999856948853, test acurracy=0.8711770176887512
epoch=10, train accuracy=0.949999988079071, test acurracy=0.8767377138137817
epoch=11, train accuracy=0.8700000047683716, test acurracy=0.8804448843002319
epoch=12, train accuracy=0.8600000143051147, test acurracy=0.8813716173171997
epoch=13, train accuracy=0.8999999761581421, test acurracy=0.881371617317199